In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Debugging and Optimizing Agents: A Guide to Tracing in Reasoning Engine

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tracing_agents_in_reasoning_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) helps you build and deploy agent-based AI applications that use LLMs and custom tools. Understanding your agent's decision-making process is essential for debugging and optimization, and [Cloud Trace](https://cloud.google.com/trace) is a great tool for exploring this tracing data to get insights.

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/reasoning-engine/images/cloud-trace-agent.png">

This notebook demonstrates how to:

- **Learn Key Concepts**: Learn about the fundamental building blocks of tracing.
- **Deploy Your Agent**: Make your tracing-enabled agent available in a production-like environment on Reasoning Engine.
- **Enable Tracing**: Enable tracing in a simple agent
- **Examine Traces**: Use the Cloud Console and Cloud Trace SDK to access and analyze a specific trace.

By the end of this notebook, you'll be able to leverage tracing to build more robust and efficient AI agents on Vertex AI.

## Concepts

Here are some of the key concepts and terminology related to tracing, which will be helpful to understand as we explore traces generated by an agent in Reasoning Engine:

Below is an example of a trace in JSON format, showing a single span. This span represents a call to a large language model (LLM). Notice how the trace data captures important details:

### Example trace

```json
{
   "name": "llm",
   "context": {
       "trace_id": "ed7b336d-e71a-46f0-a334-5f2e87cb6cfc",
       "span_id": "ad67332a-38bd-428e-9f62-538ba2fa90d4"
   },
   "span_kind": "LLM",
   "parent_id": "f89ebb7c-10f6-4bf8-8a74-57324d2556ef",
   "start_time": "2023-09-07T12:54:47.597121-06:00",
   "end_time": "2023-09-07T12:54:49.321811-06:00",
   "status_code": "OK",
   "status_message": "",
   "attributes": {
       "llm.input_messages": [
           {
               "message.role": "system",
               "message.content": "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."
           },
           {
               "message.role": "user",
               "message.content": "Hello?"
           }
       ],
       "output.value": "assistant: Yes I am here",
       "output.mime_type": "text/plain"
   },
   "events": [],
}
```

### Trace

You can think of a [trace](https://opentelemetry.io/docs/concepts/signals/traces/) like a timeline of requests as they travel through your application. A trace is composed of individual spans, with the first span representing the overall request. Each span provides details about a specific operation within the request.

### Span

A [span](https://opentelemetry.io/docs/concepts/signals/traces/#spans) represents a single unit of work, like a function call or an interaction with an LLM. It captures information such as the operation's name, start and end times, and any relevant attributes (metadata). Spans can be nested, showing parent-child relationships between operations.

### Span Attribute

[Span attributes](https://opentelemetry.io/docs/concepts/signals/traces/#attributes) are key-value pairs that provide additional context about a span. For instance, an LLM span might have attributes like the model name, prompt text, and token count.

### Span Kind

[Span kind](https://opentelemetry.io/docs/concepts/signals/traces/#span-kind) categorizes the type of operation a span represents. Common kinds include:

- `CHAIN`: Links between LLM application steps or the start of a request.
- `LLM`: A call to a large language model.
- `TOOL`: An interaction with an external tool (API, database, etc.).
- `AGENT`: A reasoning block that combines LLM and tool interactions.

## Get started

### Install Vertex AI SDK and other required packages


In [2]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    google-cloud-trace


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "mktg-dsc-prod-f0xt"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://dsc_vertex_re_staging"  # @param {type:"string"}

import vertexai


vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

## Build and deploy an agent

Let's dive into building a simple agent that utilizes tracing. This agent will use a few custom tools to demonstrate how tracing can provide insights into its workflow.

### Import libraries

Before you start building your agent, you'll import the necessary libraries. These include the Vertex AI SDK, pandas for data analysis, and the Cloud Trace SDK for working with trace data.

In [3]:
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
from google.cloud import trace_v1 as trace
from vertexai.preview import reasoning_engines
from vertexai.reasoning_engines._reasoning_engines import _utils

### Define tools

You'll define a few Python functions to act as tools for your agent. These tools will simulate actions or API calls that a real-world agent might perform. For this example, you'll create tools to classify a customer support ticket, query a knowledge base, and escalate a ticket to a human agent.

In [5]:
def classify_ticket(ticket_text: str) -> str:
    """Classifies a support ticket into a category."""
    # Simulate a call to a classification model
    categories = {
        "general": "Questions and information",
        "billing": "Payment and invoices",
        "technical": "API and SDK developer documentation",
    }
    if "payment" in ticket_text:
        category = "billing"
        description = categories[category]
    elif "settings" in ticket_text:
        category = "technical"
        description = categories[category]
    else:
        category = "general"
        description = categories[category]

    return f"This ticket is in the {category} category for questions related to {description}"


def search_knowledge_base(category: str) -> List[Dict]:
    """Searches a knowledge base for relevant articles and documentation links."""
    # Simulate a knowledge base search
    articles = {
        "general": [
            {
                "title": "Contacting support",
                "url": "https://example.com/contact",
            }
        ],
        "billing": [
            {
                "title": "How to update your payment information",
                "url": "https://example.com/billing/update",
            },
        ],
        "technical": [
            {
                "title": "Troubleshooting common login issues",
                "url": "https://example.com/technical/help",
            },
        ],
    }
    return articles.get(category, [])


def escalate_to_human(ticket_text: str) -> str:
    """Initiates escalation to a human agent for outage reports."""
    return "Your ticket has been escalated to a human agent. Please expect a response within 1-2 hours."

### Define agent and enable tracing

Now, let's define your agent using the LangChain template in Reasoning Engine and the Vertex AI SDK. Enable tracing by setting the `enable_tracing` parameter to `True`, which allows you to capture detailed information about the agent's execution.

In [7]:
agent = reasoning_engines.LangchainAgent(
    model="gemini-1.5-pro-001",
    model_kwargs={"temperature": 0},
    tools=[classify_ticket, search_knowledge_base, escalate_to_human],
    enable_tracing=True,
)

### Test your agent locally (with traces!)

Let's test your agent locally by sending it a query. Since you've enabled tracing, you'll be able to see how the agent processes this request and interacts with its tools.

In [8]:
agent.query(
    input="""
    Classify the following ticket into a category and give me a relevant documentation link.
    
    Support ticket text:
    I need to update my billing information since my payment method has expired.
    """
)

{'input': '\n    Classify the following ticket into a category and give me a relevant documentation link.\n    \n    Support ticket text:\n    I need to update my billing information since my payment method has expired.\n    ',
 'output': "This ticket is about billing. Here's a link to our documentation about updating your payment information: https://example.com/billing/update. \n"}

### Get your first trace

Before diving deeper into trace analysis, let's use the Cloud Trace SDK to retrieve a specific trace generated by your local agent. This will give you a concrete example to work with.

In [4]:
client = trace.TraceServiceClient()

I0000 00:00:1724930959.273690  307823 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1724930959.294717  307823 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            filter="openinference.span.kind:AGENT",
        )
    )
]

In [6]:
trace_data = client.get_trace(project_id=PROJECT_ID, trace_id=result[0].trace_id).spans[
    0
]
trace_data

span_id: 9129889997696345964
name: "AgentExecutor"
start_time {
  seconds: 1724929315
  nanos: 561400832
}
end_time {
  seconds: 1724929490
  nanos: 858717952
}
labels {
  key: "output.value"
  value: "```json\n{\"tool_code\": \"print(default_api.get_domain(company_name=\'LivCor\'))\", \"tool_name\": \"get_domain\", \"tool_input\": {\"company_name\": \"LivCor\"}, \"thought\": \"I need to get the domain name for LivCor so I can look up the company details. I\'ll use the get_d"
}
labels {
  key: "openinference.span.kind"
  value: "AGENT"
}
labels {
  key: "input.value"
  value: "You are an expert data analyst. Use the information collected from the tools to present your response.\n\n            You have been given access to a get_domain tool. Please use this tool to gather information about the domain if the the query references fin"
}
labels {
  key: "g.co/agent"
  value: "opentelemetry-python 1.26.0; google-cloud-trace-exporter 1.6.0"
}

After you deploy your agent and make remote queries in the following sections, you'll dive into the details for working with trace data in the Cloud Console or using the Python SDK for Cloud Trace.

### Deploy your agent

Now that you've seen how tracing works locally, let's deploy your agent to Reasoning Engine. This will allow you to send it queries in a production-like environment and observe its behavior through traces.

In [12]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
    ],
)

Using bucket your-bucket-name
Writing to gs://your-bucket-name/reasoning_engine/reasoning_engine.pkl
Writing to gs://your-bucket-name/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://your-bucket-name/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/your-project-number/locations/us-central1/reasoningEngines/3146221736555446272/operations/2750583127503011840
ReasoningEngine created. Resource name: projects/your-project-number/locations/us-central1/reasoningEngines/3146221736555446272
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/your-project-number/locations/us-central1/reasoningEngines/3146221736555446272')


### Query your deployed agent

With your agent deployed, you can interact with it remotely. Let's send a query and generate some trace data to explore.

In [13]:
remote_agent.query(
    input="""
    Classify the following ticket into a category and route the customer accordingly:
    
    Support ticket text:
    I am unable to make any API calls and I need to report an outage in the system
    """,
)

{'input': '\n    Classify the following ticket into a category and route the customer accordingly:\n    \n    Support ticket text:\n    I am unable to make any API calls and I need to report an outage in the system\n    ',
 'output': 'The ticket has been escalated to a human agent. Please expect a response within 1-2 hours. \n'}

## Exploring traces in the Cloud Console

The Cloud Trace console provides a powerful and intuitive visual interface for exploring trace data, including visualizing, filtering, and analyzing your traces.

Accessing the Trace Console:

- **Project-Level View**: To see all traces for your Google Cloud project (replace `your-project-id` with your actual project ID), go to: https://console.cloud.google.com/traces/list?project=your-project-id

- **Specific Trace**: If you know the unique Trace ID for a specific trace you want to examine, you can view it directly (replace your-trace-id with the actual Trace ID): https://console.cloud.google.com/traces/list?project=your-project-id&tid=your-trace-id

Features to Explore in the Console:

- **Trace List**: View a list of traces, sorted by start time, along with summary information (duration, number of spans).
- **Waterfall View**: Visualize the spans within a trace as a timeline, showing the duration of each operation and their relationships.
- **Span Details**: Click on a span to view its attributes, including the input and output data, and any custom metadata you've added.
- **Filtering and Search**: The console provides powerful options for filtering traces by time range, service, span name, and other criteria. You can also search for specific traces using keywords or attributes.

For a detailed guide to working with traces in the console, refer to the [Cloud Trace documentation on finding traces](https://cloud.google.com/trace/docs/finding-traces). Experiment with the Cloud Trace console to gain a deeper understanding of your agent's behavior and how it's executing within Reasoning Engine.

## Working with traces using `pandas`

For more programmatic analysis, you can use the pandas library to work with trace data. You'll fetch traces, convert them to DataFrames, and then use pandas' functionality to explore the trace data.

In [7]:
result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            filter="openinference.span.kind:AGENT",
        )
    )
]

In [8]:
trace_data = client.get_trace(project_id=PROJECT_ID, trace_id=result[0].trace_id)

In [9]:
spans = pd.DataFrame.from_records([_utils.to_dict(span) for span in trace_data.spans])
spans.head()

,spanId,name,startTime,endTime,labels,parentSpanId
0,9129889997696345964,AgentExecutor,2024-08-29T11:01:55.561400832Z,2024-08-29T11:04:50.858717952Z,{'g.co/agent': 'opentelemetry-python 1.26.0; g...,NaN
1,16981390168131386878,RunnableSequence,2024-08-29T11:01:55.584172032Z,2024-08-29T11:02:24.718503936Z,{'g.co/agent': 'opentelemetry-python 1.26.0; g...,9129889997696345964
2,14465681810097984101,"RunnableParallel<input,agent_scratchpad>",2024-08-29T11:01:55.610887168Z,2024-08-29T11:01:55.784389120Z,{'g.co/agent': 'opentelemetry-python 1.26.0; g...,16981390168131386878
3,1139659899946443699,ChatPromptTemplate,2024-08-29T11:01:55.873931008Z,2024-08-29T11:01:55.874848Z,{'g.co/agent': 'opentelemetry-python 1.26.0; g...,16981390168131386878
4,6921102655131412116,get_domain,2024-08-29T11:02:24.795828224Z,2024-08-29T11:02:25.065630976Z,{'g.co/agent': 'opentelemetry-python 1.26.0; g...,9129889997696345964


In [21]:
spans[spans["name"] == "ChatVertexAI"]


,spanId,name,startTime,endTime,labels,parentSpanId
9,3877859571125750842,ChatVertexAI,2024-08-27T16:59:24.089795072Z,2024-08-27T16:59:27.840190976Z,"{'llm.invocation_parameters': '{""model_name"": ...",10621815106830957606
14,5570406937849240747,ChatVertexAI,2024-08-27T16:59:28.454256128Z,2024-08-27T16:59:33.128314112Z,{'llm.input_messages.1.message.role': 'assista...,10435659226710918420
22,7069817728099366601,ChatVertexAI,2024-08-27T16:59:33.734412800Z,2024-08-27T16:59:36.391567104Z,"{'llm.invocation_parameters': '{""model_name"": ...",3694277093980964884


In [22]:
spans[spans["name"] == "ChatVertexAI"].labels.apply(pd.Series)

,llm.invocation_parameters,g.co/agent,metadata,output.mime_type,llm.input_messages.0.message.role,llm.token_count.total,output.value,llm.input_messages.0.message.content,llm.output_messages.0.message.function_call_arguments_json,input.mime_type,...,llm.input_messages.1.message.content,llm.input_messages.2.message.role,llm.input_messages.1.message.function_call_arguments_json,llm.input_messages.2.message.content,llm.input_messages.1.message.function_call_name,llm.input_messages.4.message.content,llm.input_messages.3.message.role,llm.input_messages.4.message.role,llm.input_messages.3.message.function_call_name,llm.input_messages.3.message.function_call_arguments_json
9,"{""model_name"": ""gemini-1.5-pro-001"", ""candidat...",opentelemetry-python 1.26.0; google-cloud-trac...,"{""ls_provider"": ""google_vertexai"", ""ls_model_n...",application/json,user,235,"{""generations"": [[{""text"": ""I can't answer all...",The domain name for Geico? What is the account...,"{""company_name"": ""Geico""}",application/json,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,"{""model_name"": ""gemini-1.5-pro-001"", ""candidat...",opentelemetry-python 1.26.0; google-cloud-trac...,"{""ls_provider"": ""google_vertexai"", ""ls_model_n...",application/json,user,394,"{""generations"": [[{""text"": """", ""generation_inf...",The domain name for Geico? What is the account...,"{""domain"": ""geico.com""}",application/json,...,I can't answer all of your questions. The avai...,tool,"{""company_name"": ""Geico""}","{""company_name"": ""GEICO"", ""company_legal_name""...",get_domain,NaN,NaN,NaN,NaN,NaN
22,"{""model_name"": ""gemini-1.5-pro-001"", ""candidat...",opentelemetry-python 1.26.0; google-cloud-trac...,"{""ls_provider"": ""google_vertexai"", ""ls_model_n...",application/json,user,483,"{""generations"": [[{""text"": ""Unfortunately, the...",The domain name for Geico? What is the account...,NaN,application/json,...,I can't answer all of your questions. The avai...,tool,"{""company_name"": ""Geico""}","{""company_name"": ""GEICO"", ""company_legal_name""...",get_domain,"{""fault"": {""faultstring"": ""Access Denied for c...",assistant,tool,get_company_details,"{""domain"": ""geico.com""}"


## Exploring traces with the Python SDK for Cloud Trace

The Cloud Trace Python SDK provides even more flexibility for working with trace data. We'll use it to demonstrate how to filter traces by date, time, labels, and view types.

**Filter by date and time**

In [11]:
# Calculate the start and end times
now = datetime.utcnow()
yesterday = now - timedelta(hours=24)

# Format the dates as ISO 8601 strings with 'Z' for UTC
end_time = now.isoformat() + "Z"
start_time = yesterday.isoformat() + "Z"

# Request a filtered list of traces by date and time
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        start_time=start_time,
        end_time=end_time,
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

project_id: "mktg-dsc-prod-f0xt"
trace_id: "0a004c629c93fab3c4c508bb5b051ba9"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "2dd94adf2d7bd3af34bdf6665cfa2d73"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "351780f97adead64af9395cc610e8922"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "528bbfb31ebee09f81e66746a8b236b5"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "5fb58c562048888577f83f2a6a4a96d8"



**Filter by label**

In [12]:
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        # Return traces where any root span's name starts with AgentExecutor
        filter="root:AgentExecutor",
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

project_id: "mktg-dsc-prod-f0xt"
trace_id: "0a004c629c93fab3c4c508bb5b051ba9"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "351780f97adead64af9395cc610e8922"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "528bbfb31ebee09f81e66746a8b236b5"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "795a0e05f740de74cff111064f49d25c"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "823c8c297f2132f088078625269d8865"



**Filter by view type**

In [10]:
result = client.list_traces(
    request=trace.types.ListTracesRequest(
        project_id=PROJECT_ID,
        # view=trace.types.ListTracesRequest.ViewType.ROOTSPAN,
        view=trace.types.ListTracesRequest.ViewType.MINIMAL,
        # view=trace.types.ListTracesRequest.ViewType.COMPLETE,
    )
)

for count, r in enumerate(result):
    if count >= 5:
        break
    print(r)

project_id: "mktg-dsc-prod-f0xt"
trace_id: "1743dacf888f7a6cdd3bba78d9d6359d"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "1fb0d7d60aee87be59427c0b8673870b"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "261784d3e4fed07048ff9181975755e3"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "69ad22d4bbfe2928ba6bef39c14cc5cf"

project_id: "mktg-dsc-prod-f0xt"
trace_id: "b9e11986887daea71a5c541dfc512059"



## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Reasoning Engine instance to avoid any unexpected charges on your Google Cloud account.

In [23]:
remote_agent.delete()

NameError: name 'remote_agent' is not defined